In [1]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
from model_src.feedbackcontroller import FeedbackController
from model_src.feedbackcontrollerTorch import FeedBackControllerTorch
torch.set_default_dtype(torch.float64)
tf.keras.backend.set_floatx('float64')
tf.keras.backend.floatx()


'float64'

In [2]:
class controller_torch(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.units               = 32
        self.output_dims         = 7
        self.basis_functions     = 11
        self.ptgloabl_units      = 42
        self.controller = FeedBackControllerTorch(
            robot_state_size=self.units,
            dimensions=self.output_dims,
            basis_functions=self.basis_functions,
            cnfeatures_size=self.units + self.output_dims + 5,
            use_LSTM = False
        )

    def forward(self, robot, initial_state, cfeatures, dmp_dt):
        return self.controller(seq_inputs=robot, states=initial_state, constants=(cfeatures, dmp_dt), training=False)

In [3]:
class controller_tf(tf.keras.Model):
    def __init__(self) -> None:
        super().__init__()
        self.units               = 32
        self.output_dims         = 7
        self.basis_functions     = 11
        self.controller = tf.keras.layers.RNN(
                    FeedbackController(
                        robot_state_size = self.units, 
                        rnn_state_size   = (tf.TensorShape([self.output_dims]), tf.TensorShape([self.units])),
                        dimensions       = self.output_dims, 
                        basis_functions  = self.basis_functions,
                        special          = None
                    ), 
                return_sequences=True)
    def forward(self, robot, cfeatures, dmp_dt, initial_state, training):
        return self.controller(inputs=robot, constants=(cfeatures, dmp_dt), initial_state=initial_state, training=training)



In [4]:
seq_len = 5
bs = 4
robot = np.random.rand(bs,seq_len,7)
cfeatures = np.random.rand(bs,44)
dmp_dt = np.random.rand(bs,1)
init_state1 = np.random.rand(bs,7)
init_state2 = np.random.rand(bs,32)

robot_tf = tf.convert_to_tensor(robot, dtype=tf.float64)
cfeatures_tf = tf.convert_to_tensor(cfeatures, dtype=tf.float64)
dmp_dt_tf = tf.convert_to_tensor(dmp_dt, dtype=tf.float64)
init_state1_tf = tf.convert_to_tensor(init_state1, dtype=tf.float64)
init_state2_tf = tf.convert_to_tensor(init_state2, dtype=tf.float64)

robot_torch = torch.tensor(robot, dtype=torch.float64)
cfeatures_torch = torch.tensor(cfeatures, dtype=torch.float64)
dmp_dt_torch = torch.tensor(dmp_dt, dtype=torch.float64)
init_state1_torch = torch.tensor(init_state1, dtype=torch.float64)
init_state2_torch = torch.tensor(init_state2, dtype=torch.float64)

ControllerTorch = controller_torch()

ControllerTF = controller_tf()


In [5]:
import pickle
with open('controller.pkl', 'wb') as f:
    pickle.dump(ControllerTorch.state_dict(), f)

In [6]:
def load_controller_torch(ct):
    with open('controller.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
    ct.load_state_dict(loaded_dict, strict=True)
    return ct

In [7]:
def load_controller_tf(ctf):
    with open('controller.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
    loaded_dict['controller.Cell.robot_gru.bias_ih'] = torch.cat((loaded_dict['controller.Cell.robot_gru.bias_hh'].unsqueeze(1),
    loaded_dict['controller.Cell.robot_gru.bias_ih'].unsqueeze(1)), dim=1)
    del loaded_dict['controller.Cell.robot_gru.bias_hh']
    del loaded_dict['controller.Cell.basismodel._centers']
    #for v, vitem in loaded_dict.items():
    #    print(v)
    #    print(vitem.shape)
    weights = [
        tf.convert_to_tensor(wt.T.numpy(), dtype=tf.float64) for wt in loaded_dict.values()
    ]
    ctf.set_weights(weights)
    for w in ctf.trainable_variables:
        print(w.dtype)
    return ctf

In [8]:
result_tf = ControllerTF.forward(robot=robot_tf, cfeatures=cfeatures_tf, dmp_dt = dmp_dt_tf, initial_state=[init_state1_tf, init_state2_tf], training=False)
ControllerTF = load_controller_tf(ControllerTF)
print('______________________________________________________________________________________________')
result_tf = ControllerTF.forward(robot=robot_tf, cfeatures=cfeatures_tf, dmp_dt = dmp_dt_tf, initial_state=[init_state1_tf, init_state2_tf], training=False)


in_robot: [0.82218201 0.3123853  0.83037212 0.60336368 0.73380905]
st_gru_last: [0.66942109 0.99952977 0.95571641 0.67223351 0.66131527]
e_robot: [0.40176806 0.26355879 0.62063157 0.33986366 0.10169596]
in_robot: [0.82218201 0.3123853  0.83037212 0.60336368 0.73380905]
st_gru_last: [0.66942109 0.99952977 0.95571641 0.67223351 0.66131527]
e_robot: [0.40176806 0.26355879 0.62063157 0.33986366 0.10169596]
in_robot: [0.24110608 0.54022686 0.90250058 0.90493803 0.89526957]
st_gru_last: [0.40176806 0.26355879 0.62063157 0.33986366 0.10169596]
e_robot: [ 0.25264624 -0.04316009  0.35226518  0.21102095 -0.13877481]
in_robot: [0.84185649 0.30061571 0.21570727 0.08532576 0.52952407]
st_gru_last: [ 0.25264624 -0.04316009  0.35226518  0.21102095 -0.13877481]
e_robot: [ 0.07580221 -0.18892998  0.24223129  0.12041768 -0.22851742]
in_robot: [0.20076891 0.83463747 0.78606288 0.78175762 0.00946002]
st_gru_last: [ 0.07580221 -0.18892998  0.24223129  0.12041768 -0.22851742]
e_robot: [-0.06603181 -0.248482

In [9]:
result_torch = ControllerTorch(robot=robot_torch, cfeatures=cfeatures_torch, dmp_dt = dmp_dt_torch, initial_state=[init_state1_torch, init_state2_torch])
print('_________________________________________________________________________________________')
ControllerTorch = load_controller_torch(ControllerTorch)
result_torch = ControllerTorch(robot=robot_torch, cfeatures=cfeatures_torch, dmp_dt = dmp_dt_torch, initial_state=[init_state1_torch, init_state2_torch])


in_robot: tensor([0.8222, 0.3124, 0.8304, 0.6034, 0.7338])
st_gru_last: tensor([0.6694, 0.9995, 0.9557, 0.6722, 0.6613])
gru_output: tensor([0.2326, 0.6443, 0.6770, 0.3773, 0.4560], grad_fn=<SliceBackward0>)
in_robot: tensor([0.2411, 0.5402, 0.9025, 0.9049, 0.8953])
st_gru_last: tensor([0.2326, 0.6443, 0.6770, 0.3773, 0.4560], grad_fn=<SliceBackward0>)
gru_output: tensor([0.0810, 0.4423, 0.5295, 0.2764, 0.3354], grad_fn=<SliceBackward0>)
in_robot: tensor([0.8419, 0.3006, 0.2157, 0.0853, 0.5295])
st_gru_last: tensor([0.0810, 0.4423, 0.5295, 0.2764, 0.3354], grad_fn=<SliceBackward0>)
gru_output: tensor([-0.0658,  0.4914,  0.2962,  0.2533,  0.2110], grad_fn=<SliceBackward0>)
in_robot: tensor([0.2008, 0.8346, 0.7861, 0.7818, 0.0095])
st_gru_last: tensor([-0.0658,  0.4914,  0.2962,  0.2533,  0.2110], grad_fn=<SliceBackward0>)
gru_output: tensor([-0.1527,  0.3895,  0.2468,  0.2224,  0.1917], grad_fn=<SliceBackward0>)
in_robot: tensor([0.4428, 0.6050, 0.3094, 0.4307, 0.1658])
st_gru_last: ten

In [10]:
result_torch[1][0,:10]

tensor([[0.4722],
        [0.4713],
        [0.4752],
        [0.4739],
        [0.4756]], grad_fn=<SliceBackward0>)

In [11]:
load_controller_tf(1)

AttributeError: 'int' object has no attribute 'set_weights'

In [4]:
for v in ControllerTF.trainable_variables:
    print(v.name)
    print(v.shape)

rnn/gru_cell/kernel:0
(7, 96)
rnn/gru_cell/recurrent_kernel:0
(32, 96)
rnn/gru_cell/bias:0
(2, 96)
rnn/dense/kernel:0
(76, 77)
rnn/dense/bias:0
(77,)
rnn/dense_1/kernel:0
(77, 77)
rnn/dense_1/bias:0
(77,)
rnn/dense_2/kernel:0
(77, 77)
rnn/dense_2/bias:0
(77,)
rnn/dense_3/kernel:0
(76, 16)
rnn/dense_3/bias:0
(16,)
rnn/dense_4/kernel:0
(16, 1)
rnn/dense_4/bias:0
(1,)
